In [16]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
from sgp4.api import Satrec, WGS72
from sgp4.conveniences import jday_datetime
from datetime import datetime, timezone

pd.set_option('display.max_columns', 500)

In [17]:
# Load all the norads used in training
train_norad_df = pd.read_pickle( os.environ['my_home_path'] + '/data/split_by_norad/train_norads.pkl.gz')
train_norad_list = train_norad_df.norad.to_list()

In [18]:
#csv_store_path = os.environ['GP_HIST_PATH']
csv_store_path = os.environ['my_home_path'] + '/data/space-track-gp-hist-sample'

necessary_columns = ['NORAD_CAT_ID','OBJECT_TYPE','OBJECT_NAME','TLE_LINE1','TLE_LINE2',
                     'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE',
                     'ECCENTRICITY', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION', 'EPOCH']

dfs = None
files = sorted([x for x in os.listdir(f'{csv_store_path}/') if x.endswith(".csv.gz")])
for f in tqdm(files):
    #df = pd.read_csv(f'{csv_store_path}/{f}', parse_dates=['EPOCH'], infer_datetime_format=True, index_col='EPOCH', compression='gzip')
    df = pd.read_csv(f'{csv_store_path}/{f}', parse_dates=['EPOCH'], infer_datetime_format=True, compression='gzip')
    # LEO = Mean Motion > 11.25 and Eccentricity < 0.25
    #df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25)]
    df = df[df.NORAD_CAT_ID.isin(train_norad_list)][necessary_columns]

    # Since animated gabbard diagrams are generated per frame, we can revert the scaling when we plot the graphs
    if dfs is None:
        dfs = df
    elif len(df) > 0:
        dfs = pd.concat([dfs,df])
            
# Remove unnecessary columns to save memory
# unnecessary_columns = ['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR', 'OBJECT_NAME', 'OBJECT_ID',
#                        'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'EPHEMERIS_TYPE',
#                        'CLASSIFICATION_TYPE', 'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'SEMIMAJOR_AXIS', 'PERIOD',
#                        'APOAPSIS', 'PERIAPSIS', 'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE',
#                        'SITE', 'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'index']
# dfs = dfs.reset_index().drop(columns=unnecessary_columns, axis=1)
dfs = dfs.reset_index()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.37it/s]


In [4]:
def get_satellite_position_data():
    # Create the satellite object (used to find satellite position)
    dfs['satobj'] = dfs.apply(lambda x: Satrec.twoline2rv(x['TLE_LINE1'], x['TLE_LINE2']), axis=1)

    # Get the Julian date of the EPOCH
    #dfs['epoch_julian'] = dfs['EPOCH'].apply(lambda x: jday_datetime(x.replace(tzinfo=timezone.utc)))
    dfs[['epoch_jd', 'epoch_fr']] = dfs['EPOCH'].apply(lambda x: jday_datetime(x.replace(tzinfo=timezone.utc))).to_list()

    # Get the days since 1949 December 31 00:00 UT
    # This will be used when creating satobj for the test set
    # (this is needed to get the satellite position from generated TLEs
    #  because of how Satrec sgp4init() works')
    ref_date = datetime.strptime('12/31/1949 00:00:00', '%m/%d/%Y %H:%M:%S')
    dfs['epoch_days'] = dfs['EPOCH'].apply(lambda x: (x-ref_date)/np.timedelta64(1, 'D'))

    # Get satellite x,y,z positions from TLE
    #dfs['satpos'] = dfs.apply(lambda x: x['satobj'].sgp4(*x['epoch_julian'])[1], axis=1)
    dfs['satpos'] = dfs.apply(lambda x: np.array(x['satobj'].sgp4(x['epoch_jd'], x['epoch_fr'])[1]), axis=1)

get_satellite_position_data()

In [5]:
def create_xy():
    
    # ML Structure
    # Input:
    #  - Reference TLE Data (+ EPOCH)
    #  - Target EPOCH
    # Output:
    #  - Target TLE Data
    
    def groups(lst):
        arr = lst.copy()
        np.random.shuffle(arr)
        i=1
        if len(lst)<=1:
            return
        while True:
            if i==len(lst):
                yield tuple((arr[i-1],arr[0]))
                break
            else:
                yield tuple((arr[i-1],arr[i]))
                i+=1
    
    # For each unique NORAD, find all TLE indexes and generate
    # a list of combinations
    idx_pairs = []
    for norad in dfs['NORAD_CAT_ID'].unique():
        norad_idxs = dfs[dfs['NORAD_CAT_ID']==norad].index.values
        if len(norad_idxs > 1):
            idx_pairs.extend(groups(norad_idxs))
    idx_pairs = np.array(idx_pairs)
    
    # Build our X/Y datasets
    X_all = dfs.loc[idx_pairs[:,0]].reset_index()
    Y_all = dfs.loc[idx_pairs[:,1]].reset_index()
    
    # This will be the column that links x and y
    key_columns = ['epoch_jd', 'epoch_fr']
    target_columns = ['target_epoch_jd', 'target_epoch_fr']
    X_all[target_columns] = Y_all[key_columns]
    
    return X_all, Y_all

X_all, y_all = create_xy()

In [6]:
def clean_X(X):
    # Perform any cleaning of values
    
    # Return only necessary columns
    X_columns = ['MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE',
                 'ECCENTRICITY', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION',
                 'epoch_jd', 'epoch_fr', 'target_epoch_jd', 'target_epoch_fr']
    
    return X[X_columns]
    
X_all_clean = clean_X(X_all)

In [7]:
def clean_Y(Y):
    # Perform any cleaning of values
    
    # Return only necessary columns
    Y_columns = ['MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'BSTAR', 'INCLINATION', 'RA_OF_ASC_NODE',
                 'ECCENTRICITY', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'MEAN_MOTION',
                 'epoch_days', 'epoch_jd', 'epoch_fr', 'satpos']
    
    return Y[Y_columns]
    
y_all_clean = clean_Y(y_all)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all_clean, y_all_clean, test_size=0.33, random_state=42, shuffle=True)

# Remove non-training columns from y_train
non_training_cols = ['epoch_days', 'epoch_jd', 'epoch_fr', 'satpos']
y_train = y_train.drop(columns=non_training_cols, axis=1)

In [9]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

y_test = y_test.drop(columns=non_training_cols, axis=1)

ESTIMATORS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10, random_state=0, n_jobs=-1),
    "Random Forest": RandomForestRegressor(n_estimators=10, random_state=0, n_jobs=-1),
    #"GradientBoost": MultiOutputRegressor(GradientBoostingRegressor(n_estimators=10, random_state=0)),
    "K-nn": KNeighborsRegressor(n_jobs=-1),
    "Linear regression": LinearRegression(n_jobs=-1),
    #"Ridge": RidgeCV(),
    #"SVM": MultiOutputRegressor(SVR()),
}

# Let's see how it does on the same NORAD
for name, estimator in ESTIMATORS.items():
    estimator.fit(X_train, y_train)
    score = estimator.score(X_test, y_test)
    print (f'{name} got score {score}')

Extra trees got score 0.6115793089284733
Random Forest got score 0.5703758460606542
K-nn got score 0.5065872838459986
Linear regression got score 0.3948084251322881


iterator.combinations()
```
Extra trees got score 0.9080013082091861
Random Forest got score 0.883921000656464
GradientBoost got score 0.5516253957176365
K-nn got score 0.7644015286756071
Linear regression got score 0.4746575589454271
Ridge got score 0.4744900285859706
```
groups()
```
Extra trees got score 0.534262241603748
Random Forest got score 0.4692323383272387
GradientBoost got score 0.356691868881702
K-nn got score 0.36862432289895664
Linear regression got score 0.4765033608802853
Ridge got score 0.45090731103313353
```

In [29]:
X_train.iloc[0]

MEAN_MOTION_DOT     -4.000000e-08
MEAN_MOTION_DDOT     0.000000e+00
BSTAR                1.000000e-04
INCLINATION          7.406290e+01
RA_OF_ASC_NODE       1.151074e+02
ECCENTRICITY         1.082020e-02
ARG_OF_PERICENTER    2.546340e+02
MEAN_ANOMALY         1.042746e+02
MEAN_MOTION          1.227767e+01
epoch_jd             2.452040e+06
epoch_fr             9.721857e-01
target_epoch_jd      2.452042e+06
target_epoch_fr      9.498902e-01
Name: 111753, dtype: float64

In [141]:
torch.from_numpy(X_train.to_numpy()).float()[0].shape

torch.Size([13])

In [148]:
import torch
import torch.nn as nn
torch.manual_seed(0)

hiddenSize = 300
batchSize = 200
learningRate = 0.01
numEpochs = 10

# Help with cuda: https://discuss.pytorch.org/t/how-to-load-all-data-into-gpu-for-training/27609
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu') # 3min 46s / 3min 54s / 15.6 s
#device = torch.device('cuda') # 12.1 s
#device = torch.device('cuda:0')

# Another idea is to use parallelism instead of GPU: https://pytorch.org/tutorials/intermediate/ddp_tutorial.html

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
# for inputs, labels in trainLoader:
#     inputs = to_device(inputs, device)
#     break

# Build Dataset and DataLoader: https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, X_data, y_data, device='cpu'):
        'Initialization'
        self.y_data = to_device(torch.from_numpy(y_data.to_numpy()).float(), device)
        self.X_data = to_device(torch.from_numpy(X_data.to_numpy()).float(), device)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.X_data)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample and convert from pandas to numpy to tensor
        X = self.X_data[index]
        y = self.y_data[index]
        
        # Send to GPU
        #X = to_device(X, self.device)
        #y = to_device(y, self.device)

        return X, y
    
class NNModel(nn.Module):
    def __init__(self, inputSize, outputSize, hiddenSize, activate=None):
        super().__init__()
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.Tanh() if activate == "Tanh" else nn.ReLU()
        self.layer1 = nn.Linear(inputSize, hiddenSize)
        self.layer2 = nn.Linear(hiddenSize, outputSize)

    def forward(self, X):
        hidden = self.activate(self.layer1(X))
        return self.layer2(hidden)
        
        
net = NNModel(len(X_train.columns), len(y_train.columns), hiddenSize)
to_device(net, device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=learningRate)


trainDataset = Dataset(X_train, y_train, device)
trainLoader = torch.utils.data.DataLoader(dataset=trainDataset,
                                          batch_size=batchSize,
                                          shuffle=True,
                                          #pin_memory=True,
                                         )
# Test will be done by other means...see below
# testDataset = Dataset(X_test, y_test.iloc[:,:len(y_train.columns)], device)
# testLoader = torch.utils.data.DataLoader(dataset=testDataset,
#                                          batch_size=batchSize, shuffle=False, pin_memory=True)

In [149]:
%%time
print('>>> Beginning training!')
for epoch in range(numEpochs):
    for i, (inputs, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        # Forward propagation
        outputs = net(inputs)
        # Backpropagation
        loss = criterion(outputs, labels)
        loss.backward()
        # Gradient descent
        optimizer.step()
        # Logging
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'.format(epoch+1, numEpochs, i+1,
                                                                 len(trainDataset)//batchSize, loss))

>>> Beginning training!
Epoch [1/10], Step [100/635], Loss: 3146.41943359375
Epoch [1/10], Step [200/635], Loss: 1181.62646484375
Epoch [1/10], Step [300/635], Loss: 69.24398803710938
Epoch [1/10], Step [400/635], Loss: 69.44286346435547
Epoch [1/10], Step [500/635], Loss: 68.45832061767578
Epoch [1/10], Step [600/635], Loss: 68.99266815185547
Epoch [2/10], Step [100/635], Loss: 66.46878814697266
Epoch [2/10], Step [200/635], Loss: 67.81497192382812
Epoch [2/10], Step [300/635], Loss: 69.0251693725586
Epoch [2/10], Step [400/635], Loss: 65.7192153930664
Epoch [2/10], Step [500/635], Loss: 64.6150131225586
Epoch [2/10], Step [600/635], Loss: 66.43534088134766
Epoch [3/10], Step [100/635], Loss: 64.3507308959961
Epoch [3/10], Step [200/635], Loss: 63.36769104003906
Epoch [3/10], Step [300/635], Loss: 63.13770294189453
Epoch [3/10], Step [400/635], Loss: 62.3521728515625
Epoch [3/10], Step [500/635], Loss: 61.92041015625
Epoch [3/10], Step [600/635], Loss: 61.53953552246094
Epoch [4/10], 

In [122]:
# print('>>> Beginning validation!')
# correct, total = 0, 0
# for i, (inputs, labels) in enumerate(testLoader):
#     outputs = net(inputs)
#     _, prediction = torch.max(outputs, axis=1)
#     correct += torch.sum(prediction == labels)
#     total += labels.size(0)
# print('Validation accuracy: {}%'.format(correct.item()/total*100))

In [155]:
# Move the test set to GPU and get the results then move the results back to the CPU as numpy array
X_test_tensor = to_device(torch.from_numpy(X_test.to_numpy()).float(), device)
nn_results = net(X_test_tensor).detach().cpu().numpy()
nn_results.shape

(62617, 9)

In [156]:
dfs.loc[0,'RA_OF_ASC_NODE']

180.1561

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
#model = ExtraTreesRegressor(n_estimators=10, random_state=0, n_jobs=-1).fit(X_train, y_train)
model = LinearRegression(n_jobs=-1).fit(X_train, y_train)

In [85]:
results = model.predict(X_test)
results.shape

(62617, 9)

Numpy version of SGP4:
https://github.com/aerospaceresearch/orbitdeterminator/blob/master/orbitdeterminator/propagation/sgp4.py

In [86]:
def get_predicted_error(results):
    '''
    This returns the Mean-Squared Error
    of the predicted TLE's satellite position
    vs the actional TLE's satellites position
    '''
    
    def get_mserror(x):
        return ((x['satpos']-x['satpos_calc'])**2).mean()

    def get_satpos(x):
        sat = Satrec()
        sat.sgp4init(
             WGS72,           # gravity model
             'i',             # 'a' = old AFSPC mode, 'i' = improved mode
             0,               # satnum: Satellite number
             x['epoch_days'],       # epoch: days since 1949 December 31 00:00 UT
             x['BSTAR'],      # bstar: d`rag coefficient (/earth radii)
             x['MEAN_MOTION_DOT'], # ndot (NOT USED): ballistic coefficient (revs/day)
             x['MEAN_MOTION_DDOT'],             # nddot (NOT USED): mean motion 2nd derivative (revs/day^3)
             x['ECCENTRICITY'],       # ecco: eccentricity
             x['ARG_OF_PERICENTER'], # argpo: argument of perigee (radians)
             x['INCLINATION'], # inclo: inclination (radians)
             x['MEAN_ANOMALY'], # mo: mean anomaly (radians)
             x['MEAN_MOTION'], # no_kozai: mean motion (radians/minute)
             x['RA_OF_ASC_NODE'], # nodeo: right ascension of ascending node (radians)
        )
        return np.array(sat.sgp4(x['epoch_jd'], x['epoch_fr'])[1])

    # Join our results with the y_test column data
    y_test_error = pd.DataFrame(results, columns=y_test.columns[:-4]) \
                     .merge(y_test.reset_index()[['epoch_days', 'epoch_jd', 'epoch_fr', 'satpos']],
                            left_index=True, right_index=True)
    
    # Convert columns to radians
    cols_to_radians = ['RA_OF_ASC_NODE', 'MEAN_ANOMALY', 'INCLINATION', 'ARG_OF_PERICENTER']
    y_test_error[cols_to_radians] = y_test_error[cols_to_radians]*np.pi/180
    y_test_error['MEAN_MOTION'] = y_test_error['MEAN_MOTION']*np.pi/(4*180)
    
    # Calculate position based on predicted values
    y_test_error['satpos_calc'] = y_test_error.apply(get_satpos, axis=1)
    
    # Get the error between calculated position and TLE position
    y_test_error['pos_predict_error'] = y_test_error.apply(get_mserror, axis=1)
    
    return y_test_error

In [157]:
y_predict_error = get_predicted_error(results)
y_nn_pred_error = get_predicted_error(nn_results)
y_predict_error['pos_predict_error'].describe()

count    5.929700e+04
mean     2.536345e+07
std      5.935988e+07
min      5.760510e+01
25%      2.691235e+06
50%      1.239972e+07
75%      3.095732e+07
max      2.200150e+09
Name: pos_predict_error, dtype: float64

In [88]:
def get_propigated_error():
    '''
    This returns the Mean-Squared Error
    of the propigated TLE's satellite position
    vs the actional TLE's satellites position
    '''
    def get_satpos(x):
        return np.array(x.satobj.sgp4(x.target_epoch_jd, x.target_epoch_fr)[1])

    def get_mserror(x):
        return ((x['satpos']-x['satpos_prop'])**2).mean()

    X_propigation_error = X_all.loc[X_test.index]
    X_propigation_error['satpos_prop'] = X_propigation_error.apply(get_satpos, axis=1)
    X_propigation_error['pos_propigate_error'] = X_propigation_error.apply(get_mserror, axis=1)
    
    return X_propigation_error

X_prop_error = get_propigated_error()

In [89]:
X_prop_error['pos_propigate_error'].describe()

count    6.254500e+04
mean     3.161009e+34
std      7.905365e+36
min      2.149102e-01
25%      3.260486e+05
50%      2.411628e+06
75%      2.582770e+07
max      1.977053e+39
Name: pos_propigate_error, dtype: float64

In [166]:
print(f'Mean Error in Propigation {np.sqrt(X_prop_error.pos_propigate_error.mean())}')
print(f'Mean Error in Prediction {np.sqrt(y_predict_error.pos_predict_error.mean())}')
print(f'Mean Error in Prediction {np.sqrt(y_nn_pred_error.pos_predict_error.mean())}')

Mean Error in Propigation 1.777922663430849e+17
Mean Error in Prediction 5036.214051855435
Mean Error in Prediction 6062.998220039357


In [106]:
# Best performing satpos prediction - LinearRegression
# satpos - The x,y,z based on actual TLE and epoch (same as from y_test)
# satpos_calc - The x,y,z calculated based on the ML model
y_predict_error.sort_values(by='pos_predict_error').reset_index().iloc[0]

index                                                            59338
MEAN_MOTION_DOT                                               0.000423
MEAN_MOTION_DDOT                                              0.000003
BSTAR                                                           0.0093
INCLINATION                                                   1.718075
RA_OF_ASC_NODE                                                3.780205
ECCENTRICITY                                                  0.003082
ARG_OF_PERICENTER                                              1.97072
MEAN_ANOMALY                                                  4.320189
MEAN_MOTION                                                   0.064617
epoch_days                                                18758.633707
epoch_jd                                                     2452039.5
epoch_fr                                                      0.633707
satpos               [-5622.023417389518, -4185.879322657325, 5.324...
satpos

In [107]:
# The actual result
# satpos - The x,y,z based on actual TLE and epoch
y_test.iloc[59338]

MEAN_MOTION_DOT                                               0.000841
MEAN_MOTION_DDOT                                                   0.0
BSTAR                                                         0.011192
INCLINATION                                                    98.4392
RA_OF_ASC_NODE                                                 216.676
ECCENTRICITY                                                  0.003657
ARG_OF_PERICENTER                                              78.8616
MEAN_ANOMALY                                                  281.7141
MEAN_MOTION                                                  14.787814
epoch_days                                                18758.633707
epoch_jd                                                     2452039.5
epoch_fr                                                      0.633707
satpos               [-5622.023417389518, -4185.879322657325, 5.324...
Name: 183865, dtype: object

In [168]:
# The NN result
y_nn_pred_error.loc[59338]

MEAN_MOTION_DOT                                              -0.000811
MEAN_MOTION_DDOT                                             -0.001248
BSTAR                                                         -0.00093
INCLINATION                                                   1.036004
RA_OF_ASC_NODE                                                1.020209
ECCENTRICITY                                                  0.005052
ARG_OF_PERICENTER                                             1.011387
MEAN_ANOMALY                                                  1.011653
MEAN_MOTION                                                   0.060026
epoch_days                                                18758.633707
epoch_jd                                                     2452039.5
epoch_fr                                                      0.633707
satpos               [-5622.023417389518, -4185.879322657325, 5.324...
satpos_calc          [-4560.803892385173, -1031.3778705869129, 5647...
pos_pr

In [109]:
# The x,y,z based on propigationa
X_prop_error.iloc[59338]['satpos_prop']

array([-5642.35108646, -4142.41691223,   331.56394272])